In [1]:
# !pip install google-api-python-client
# !pip install tkcalendar


Version_details = """

Version 2:
Download attachment and add suffix as receive date---------- ok


Version 1:
Download attachment which name is in search box ------------ ok
Search using Keyword---------------------------------------- ok
Select Date Range using calendar---------------- ----------- ok
Show number of email---------------------------------------- ok

"""


import os.path
import datetime
import base64
import tkinter as tk
from tkinter import ttk
from tkcalendar import DateEntry
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/gmail.readonly"]

def download_pdf_attachments(service, start_date, end_date, keyword, attachment_name):
    """Download PDF attachments from emails containing the specified keyword between the specified start and end dates."""
    start_date_str = start_date.strftime('%Y/%m/%d')
    end_date_str = end_date.strftime('%Y/%m/%d')

    print(f"Querying emails from {start_date_str} to {end_date_str} with keyword '{keyword}'")

    try:
        # Form the query string
        query = f"after:{start_date_str} before:{end_date_str} {keyword}"
        response = service.users().messages().list(userId="me", q=query, maxResults=500).execute()
        
        # Get the list of emails
        emails = response.get("messages", [])
        num_emails = len(emails)

        # Handle pagination
        while 'nextPageToken' in response:
            page_token = response['nextPageToken']
            response = service.users().messages().list(userId="me", q=query, pageToken=page_token, maxResults=500).execute()
            emails.extend(response.get("messages", []))
            num_emails += len(response.get("messages", []))

        # Download PDF attachments
        for email in emails:
            msg = service.users().messages().get(userId="me", id=email['id']).execute()
            email_date = datetime.datetime.fromtimestamp(int(msg['internalDate']) / 1000).strftime('%Y%m%d_%H%M%S')
            for part in msg['payload']['parts']:
                if part['filename'] and part['filename'].endswith('.pdf') and attachment_name in part['filename']:
                    if 'data' in part['body']:
                        data = part['body']['data']
                    else:
                        att_id = part['body']['attachmentId']
                        att = service.users().messages().attachments().get(userId="me", messageId=email['id'], id=att_id).execute()
                        data = att['data']

                    file_data = base64.urlsafe_b64decode(data.encode('UTF-8'))
                    filename = part['filename'].rsplit('.', 1)
                    new_filename = f"{filename[0]}_{email_date}.{filename[1]}"
                    path = os.path.join('.', new_filename)

                    with open(path, 'wb') as f:
                        f.write(file_data)
                    print(f"Downloaded {new_filename}")

        return num_emails

    except HttpError as error:
        print(f"An error occurred: {error}")
        return None

def main():
    """Shows basic usage of the Gmail API.
    Lists the user's Gmail labels.
    """
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists("token.json"):
        creds = Credentials.from_authorized_user_file("token.json", SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                "credentials.json", SCOPES
            )
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open("token.json", "w") as token:
            token.write(creds.to_json())

    try:
        # Call the Gmail API
        service = build("gmail", "v1", credentials=creds)

        # Create a simple GUI to select start and end dates
        root = tk.Tk()
        root.title("Select Start and End Dates, Keyword, and Attachment Name")

        tk.Label(root, text="Select Start Date").grid(row=0, column=0)
        start_date_entry = DateEntry(root, width=12, background='darkblue',
                                     foreground='white', borderwidth=2)
        start_date_entry.grid(row=0, column=1)

        tk.Label(root, text="Select End Date").grid(row=1, column=0)
        end_date_entry = DateEntry(root, width=12, background='darkblue',
                                   foreground='white', borderwidth=2)
        end_date_entry.grid(row=1, column=1)

        tk.Label(root, text="Enter Keyword").grid(row=2, column=0)
        keyword_entry = tk.Entry(root, width=15)
        keyword_entry.grid(row=2, column=1)

        tk.Label(root, text="Enter Attachment Name").grid(row=3, column=0)
        attachment_name_entry = tk.Entry(root, width=15)
        attachment_name_entry.grid(row=3, column=1)

        def on_submit():
            start_date = start_date_entry.get_date()
            end_date = end_date_entry.get_date() + datetime.timedelta(days=1)
            keyword = keyword_entry.get().strip()
            attachment_name = attachment_name_entry.get().strip()

            num_emails = download_pdf_attachments(service, start_date, end_date, keyword, attachment_name)
            
            if num_emails is not None:
                result_label.config(text=f"Number of emails received: {num_emails}")
            else:
                result_label.config(text="Unable to retrieve the number of emails.")

        submit_button = tk.Button(root, text="Submit", command=on_submit)
        submit_button.grid(row=4, columnspan=2)

        result_label = tk.Label(root, text="")
        result_label.grid(row=5, columnspan=2)

        root.mainloop()

    except HttpError as error:
        print(f"An error occurred: {error}")

if __name__ == "__main__":
    main()


Querying emails from 2024/07/03 to 2024/07/08 with keyword 'portfolio'
Downloaded portfolio72228_20240704_210146.pdf
Downloaded portfolio72228_20240703_213802.pdf
